In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.datasets import make_classification


In [ ]:
path = "/content/drive/MyDrive/bank-additional-full.csv"
df = pd.read_csv(path, sep = ';')

In [ ]:
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [ ]:
df.replace(r'unknown', np.nan, inplace=True)

In [ ]:
df = df.drop(['duration','default','poutcome','pdays','nr.employed','euribor3m','contact','day_of_week','cons.price.idx'],axis=1)

In [ ]:
df

,age,job,marital,education,housing,loan,month,campaign,previous,emp.var.rate,cons.conf.idx,y
0,56,housemaid,married,basic.4y,no,no,may,1,0,1.1,-36.4,no
1,57,services,married,high.school,no,no,may,1,0,1.1,-36.4,no
2,37,services,married,high.school,yes,no,may,1,0,1.1,-36.4,no
3,40,admin.,married,basic.6y,no,no,may,1,0,1.1,-36.4,no
4,56,services,married,high.school,no,yes,may,1,0,1.1,-36.4,no
...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,yes,no,nov,1,0,-1.1,-50.8,yes
41184,46,blue-collar,married,professional.course,no,no,nov,1,0,-1.1,-50.8,no
41185,56,retired,married,university.degree,yes,no,nov,2,0,-1.1,-50.8,no
41186,44,technician,married,professional.course,no,no,nov,1,0,-1.1,-50.8,yes


In [ ]:
df['age'] = StandardScaler().fit_transform(df['age'].values.reshape(-1,1))


In [ ]:
df['marital'] = df['marital'].fillna(df['marital'].mode().iloc[0])
marital_enc = (df.groupby('marital').size()) / len(df)
df['marital_enc'] = df['marital'].apply(lambda x : marital_enc[x])
df.head

<bound method NDFrame.head of             age          job  marital            education housing loan month  \
0      1.533034    housemaid  married             basic.4y      no   no   may   
1      1.628993     services  married          high.school      no   no   may   
2     -0.290186     services  married          high.school     yes   no   may   
3     -0.002309       admin.  married             basic.6y      no   no   may   
4      1.533034     services  married          high.school      no  yes   may   
...         ...          ...      ...                  ...     ...  ...   ...   
41183  3.164336      retired  married  professional.course     yes   no   nov   
41184  0.573445  blue-collar  married  professional.course      no   no   nov   
41185  1.533034      retired  married    university.degree     yes   no   nov   
41186  0.381527   technician  married  professional.course      no   no   nov   
41187  3.260295      retired  married  professional.course     yes   no   nov  

In [ ]:
df['housing'] = LabelEncoder().fit_transform(df['housing'])
df['loan'] = LabelEncoder().fit_transform(df['loan'])
df['y'] = LabelEncoder().fit_transform(df['y'])
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df['education'] = imp.fit_transform(df['education'].values.reshape(-1,1))
df['education'].replace(['basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree' ],[1,2,3,4,0,6,5], inplace=True)
df['month'].replace(['may', 'jun','jul','aug','nov','jan','feb','mar','apr','sep','oct','dec' ],[3,2,2,2,2,1,1,1,1,1,1,1], inplace=True)
df['cons.conf.idx'] = StandardScaler().fit_transform(df['cons.conf.idx'].values.reshape(-1,1))
df['job'] = LabelEncoder().fit_transform(df['job'])
df = df.drop(['marital'],axis=1)



In [ ]:
df

,age,job,education,housing,loan,month,campaign,previous,emp.var.rate,cons.conf.idx,y,marital_enc
0,1.533034,3,1.0,0,0,3,1,0,1.1,0.886447,0,0.607167
1,1.628993,7,4.0,0,0,3,1,0,1.1,0.886447,0,0.607167
2,-0.290186,7,4.0,1,0,3,1,0,1.1,0.886447,0,0.607167
3,-0.002309,0,2.0,0,0,3,1,0,1.1,0.886447,0,0.607167
4,1.533034,7,4.0,0,1,3,1,0,1.1,0.886447,0,0.607167
...,...,...,...,...,...,...,...,...,...,...,...,...
41183,3.164336,5,6.0,1,0,2,1,0,-1.1,-2.224953,1,0.607167
41184,0.573445,1,6.0,0,0,2,1,0,-1.1,-2.224953,0,0.607167
41185,1.533034,5,5.0,1,0,2,2,0,-1.1,-2.224953,0,0.607167
41186,0.381527,9,6.0,0,0,2,1,0,-1.1,-2.224953,1,0.607167


In [ ]:
y = df[['y']].copy()
df = df.drop(['y'],axis=1)

# SMOTE
smote_technique = SMOTE(sampling_strategy='minority')
X_smt, y_smt = smote_technique.fit_resample(df, y)

# splitting into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X_smt, y_smt, test_size=0.2, random_state=15, stratify=y_smt)

In [ ]:
knn= KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred= knn.predict(X_test)
metrics.accuracy_score(y_test,y_pred)

/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.8367305061559508

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred  =  classifier.predict(X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y_pred

array([0, 0, 0, ..., 1, 0, 1])

In [ ]:
ac = accuracy_score(y_test,y_pred)
ac


0.7132694938440493